In [2]:
(define nil ())

# 3 Tail-Call Optimization

Tail-call optimization allows programmers to write recursive functions that use a constant amount of space. A `tail call` occurs when a function calls another function as its **last action of the current frame**; the frame is no longer needed, and thus we don't need to keep it, this way we can save memory. 

Consider the following `factorial` implementation,

In [ ]:
(define (fact n)
  (if (= n 0)
      1
      (* n (fact (- n 1)))

After calling `(fact (- n 1))`, the function still needs to multiply that result with `n`. The final expression that is evaluated is the multiplication function, not the `fact` call itself. Thus, this recursive call is **not** a tail call.

We can rewrite this function using a helper function that remembers the product that we calculated so far in each recursive step.

In [ ]:
(define (fact n)
  (define (fact-tail n result) ; Helper function
    (if (= n 0) 
        result ; If we have reached 0, then return the result
        (fact-tail (- n 1) (* n result))
  (fact-tail n 1)) ; Start with calling fact-tail with result = 1

In `fact-tail`, the recursive `fact-tail` is the last expression to be evaluated, so it is a tail call. Thus, the `fact-tail` is a tail-recursive process.

## Tail Context

When trying to identify whether a function call within the body of a function is a `tail call`, we look whether the call expression is in `tail context`.

Given that each of the following expressions is the **last expression in the body of the function**, we consider the tail context of each expression to be:
* The 2nd or 3rd operand in an `if` expression
* Any of the non-`<predicate>` sub-expressions in a `cond` expression
    * e.g. the 2nd expression of each clause
* The last operand in...
    * An `and` or `or` expression
    * A `begin` expression's body
    * A `let` expression's body

## Questions

### 3.1
For each of the following functions, identify whether it contains a recursive call in a tail context. Also indicate if it uses a constant number of frames.

In [ ]:
(define (question-a x)
  (if (= x 0) 0
      (+ x (question-a (- x 1)))

For the procedure `question-a` above, the last expression that is evaluated is a call to `+`, thus the recursive call is not in tail context. This function doesn't use a constant number of frames. Instead, this function use a linear $\Theta(n)$ frames.

In [ ]:
(define (question-b x y)
  (if (= x 0) y
      (question-b (- x 1) (+ y x))

For the procedure `question-b` above, the recursive call is the 3rd operand in the `if` expression, thus the recursive call is in a tail context. This means the last expression that will be evaluated in the procedure body is the recursive call. This function uses a constant number of frames.

In [ ]:
(define (question-c x y)
  (if (> x y)
      (question-c (- y 1) x)
      (question-c (+ x 10) y)))

For the procedure `question-c` above, the recursive calls are both the 2nd and 3rd operands of the `if` expression. Only one of these 2 recursive calls is actually evaluated, and both of them are the last expression evaluated in the body of the procedure. Therefore, the recursive calls are in a tail context, and this procedure uses a constant number of frames.

In [ ]:
(define (question-d n)
  (if (question-d n)
      (question-d (- n 1))
      (question-d (+ n 10))))

For the procedure `question-d` above, the recursive calls are both the 2nd and 3rd operands of the `if` expression. HOWEVER, the `<predicate>` of the `if` statement is a recursive call, which means the recursive call is not in a tail context! This procedure doesn't use a constant number of frames.

In [ ]:
define (question-e n)
    (cond ((= n 0) 1)
          ((question-e (- n 1)) (question-e (- n 2)))
          (else (begin (print 2) (question-e (- n 3))))

For the procedure above, the recursive calls in the 2nd and 3rd clause of the `cond` statement are in tail context. However, for the 2nd clause,

`((question-e (- n 1)) (question-e (- n 2)))`

The predicate is a recursive call, which is not in tail context. Thus, this procedure is not tail recursive and doesn't use a constant number of frames.

### 3.2
Write a tail recursive function that returns the `n`th fibonacci number. We define `fib(0) = 0` and `fib(1)` = 1

In [3]:
(define (fib n)
  (define (fib-sofar i prev current) ; i is a counter , prev is the previous value, current is the current value
    (if (= i n) ; if i has reached n, then return the current value
        current
        (fib-sofar (+ i 1) current (+ prev current))))
  (fib-sofar 1 0 1))

Above, it makes sense to start with `prev` = 0, `current` = 1 and `i` = 1. We can't start with `i` = 0 because that means we are starting with `current` = 0, and there is no fibonacci number available before the 0th fibonacci number!

### 3.3
Write a `tail recursive` function that takes in a Scheme list and returns the numerical sum of all values in the list. You can assume that the list is well-formed and contains only numbers (no nested lists).

In [10]:
(define (sum lst)
  (define (sum-helper lst current)
    (if (null? lst) current ; If we've reached the end of the list, then return the current sum
        (sum-helper (cdr lst) (+ (car lst) current)))) ; Otherwise, recursive call sum-helper
  (sum-helper lst 0))

### 3.4
Write a tail recursive function that takes in a number and a sorted list. The function returns a sorted copy with the number inserted in the correct position.

**(a)** Begin by writing a tail recursive function that reverses a list.

In [3]:
(define (reverse lst)
  (define (reverse-sofar lst lst-sofar)
    (if (null? lst) lst-sofar ; If we run out of elements in lst, then return what we have so far
        (reverse-sofar (cdr lst) (cons (car lst) lst-sofar))))
  (reverse-sofar lst nil))

Above, the following recursive call,

In [ ]:
(reverse-sofar (cdr lst) (cons (car lst) lst-sofar))

...means that for the next recursive call, the "already constructed" list so far would become the second element of the cons. Thus, the next element is inserted before the list that is already constructed.

In [2]:
; Test the reverse procedure
(reverse '(1 2 3))

(3 2 1)

**(b)** Next, write a tail recursive function that concatenates 2 lists together. You may use `reverse`.

In [4]:
(define (append a b)
  (define (rev-append-tail a b)
    (if (null? a) b
        (rev-append-tail (cdr a) (cons (car a) b))))
  (rev-append-tail (reverse a) b))

Above, the following line, 

In [ ]:
(rev-append-tail (cdr a) (cons (car a) b))

...indicates that for every recursive `rev-append-tail`, the elements of `a` will be appended in a reversed order. For example, if:
1. `a` = `(1 2 3)`
2. `b` = `(4 5 6)`

Then the result of `rev-append-tail` would eventually be `(3 2 1 4 5 6)`

In that case, one way around it is to call the `rev-append-tail` function on the reversed version of `a`.

**(c)** Finally, implement `insert`. You may use `reverse` and `append`.

In [1]:
(define (insert n lst)
  (define (rev-insert lst rev-lst)
    (cond ((null? lst) (cons n rev-lst))
          ((> (car lst) n) (append (reverse lst) (cons n rev-lst)))
          (else (rev-insert (cdr lst) (cons (car lst) rev-lst)))))
  (reverse (rev-insert lst nil)))

Note that in the implementation above, the first 2 clauses don't have recursive `rev-insert` at all. This means if any of the 2 clauses is executed, then the execution will end! The only recursive tall is the last clause of the `cond` expression.

In [5]:
; Test the insert method
(insert 3 '(1 2 4))

(1 2 3 4)